In [1]:
prefix_1 = """Q:What is the native language of Christiane Cohendy?\nA:French\n"""
prefix_3 = """Q:What is the native language of Christiane Cohendy?\nA:French\nQ:What is the final year of Atlanta Flames?\nA:1980\nQ:What is Barbara Legrand's position on the field while playing football?\nA:midfielder\n"""

In [2]:
import transformers
import torch
import json
import os
os.environ['CUDA_VISIBLE_DEVICES']= '0'
# model_name = 'Meta-Llama-3-8B-Instruct'
model_name = 'gpt-j-6b'
model_id = f"/share/huggingface/{model_name}"

In [3]:
pipeline = transformers.pipeline("text-generation", 
                                 model=model_id, 
                                 model_kwargs={"torch_dtype": torch.bfloat16}, 
                                 device_map="auto",)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# data_path = './benchmark_wiki_counterfact_test_cf.json'
data_path = './benchmark_ZsRE_ZsRE-test-all.json'
target_new_lens,ground_truth_lens = [],[]
with open(data_path, 'r') as f:
    data_list = json.load(f)

In [5]:
for data in data_list:
    target_new_lens.append(len(data['target_new'].split(' ')))
    # ground_truth_lens.append(len(data['ground_truth'].split(' ')))
    ground_truth_lens.append(len(data['ground_truth'][0].split(' ')))

sum(target_new_lens)/len(target_new_lens),\
sum(ground_truth_lens)/len(ground_truth_lens),\
max(target_new_lens),max(ground_truth_lens)

(2.026840490797546, 1.968558282208589, 8, 8)

In [6]:
with open(f'answer-zsre-{model_name}.json', 'a') as f:
    for data in data_list[:100]:
        prompt1 = data['prompt']
        prompt2 = f"{prefix_1}Q:{data['prompt']}\nA:"
        prompt3 = f"{prefix_3}Q:{data['prompt']}\nA:"
        answer1 = pipeline(prompt1, max_new_tokens=10, do_sample=False)
        answer2 = pipeline(prompt2, max_new_tokens=10, do_sample=False)
        answer3 = pipeline(prompt3, max_new_tokens=10, do_sample=False)
        data_dict = {
            'prompt': data['prompt'],
            'answer1': answer1[0]['generated_text'].replace(prompt1, ''),
            'answer2': answer2[0]['generated_text'].replace(prompt2, ''),
            'answer3': answer3[0]['generated_text'].replace(prompt3, ''),
        }

        f.write(json.dumps(data_dict, ensure_ascii=False) + '\n')
        f.flush()

/home/lyc/miniconda3/envs/ke2torch23cu121/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/lyc/miniconda3/envs/ke2torch23cu121/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128